In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 第七章 数据清洗和准备
《利用python进行数据分析 -- 第二版》

### 数据加载、清理、转换、重塑
本章讨论处理缺失数据、重复数据、字符串操作和其他分析数据转换的工具

#### 处理缺失数据
pandas一直尽量可以轻松处理缺失数据。

In [2]:
# 缺失值叫做哨兵值，用NaN表示
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
#python内置的None值在数组中也可以作为NA
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### 滤除缺失数据,方法可以有:pandas.isnull或布尔索引的手工方法、dropna

##### Series对象

In [5]:
# dropna返回一个仅含非空数据和索引值的Series
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
# 等价于 data.dropna()
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

##### DataFrame对象

In [7]:
# dropna默认丢弃任何含有缺失值的行
data = pd.DataFrame([[1.,6.5,3.],[1.,np.nan,np.nan],
                    [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [9]:
# 传入how = 'all'将只丢弃全为NA的那些行
data.dropna(how='all')                              #丢弃全为NA的行

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [10]:
# 用这种方式丢弃列
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')                   #丢弃全为NA的列  

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


###### df.dropna(thresh=2)    thresh=n 每行至少有n个非空值

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,-2.530169,NaN,NaN
1,0.455677,NaN,NaN
2,2.865351,NaN,0.412362
3,1.236643,NaN,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [13]:
df.dropna()                                 #只要有NA值，就剔除该行

,0,1,2
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [14]:
df.dropna(thresh=2)                         #每行至少n个非空值，thresh=n

,0,1,2
2,2.865351,NaN,0.412362
3,1.236643,NaN,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [15]:
df.dropna(thresh=2, axis=1)                 #每列至少n个非空值， thresh=n

,0,1,2
0,-2.530169,NaN,NaN
1,0.455677,NaN,NaN
2,2.865351,NaN,0.412362
3,1.236643,NaN,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [16]:
df.dropna(thresh=4, axis=1)                  #每列至少n个非空值， thresh=n

,0,2
0,-2.530169,NaN
1,0.455677,NaN
2,2.865351,0.412362
3,1.236643,1.287114
4,-1.077962,-0.162800
5,1.307046,-0.892294
6,0.036799,-0.402535


#### 填充缺失数据。fillna方法是最主要的函数

In [17]:
df

,0,1,2
0,-2.530169,NaN,NaN
1,0.455677,NaN,NaN
2,2.865351,NaN,0.412362
3,1.236643,NaN,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [18]:
df.fillna(0)

,0,1,2
0,-2.530169,0.000000,0.000000
1,0.455677,0.000000,0.000000
2,2.865351,0.000000,0.412362
3,1.236643,0.000000,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [19]:
df.fillna({1:0.5, 2:0})                   #通过一个字典调用fillna，实现对不同的列填充不同的值

,0,1,2
0,-2.530169,0.500000,0.000000
1,0.455677,0.500000,0.000000
2,2.865351,0.500000,0.412362
3,1.236643,0.500000,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [20]:
df.fillna(0, inplace=True)
df

,0,1,2
0,-2.530169,0.000000,0.000000
1,0.455677,0.000000,0.000000
2,2.865351,0.000000,0.412362
3,1.236643,0.000000,1.287114
4,-1.077962,0.730849,-0.162800
5,1.307046,2.110415,-0.892294
6,0.036799,-1.826710,-0.402535


In [21]:
# 对于reindexing有效的那些插值方法也可用于fillna
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,0.519170,-0.295351,0.082423
1,0.397836,-0.611371,0.587165
2,-0.925333,NaN,-0.924221
3,0.080129,NaN,1.447063
4,-0.004249,NaN,NaN
5,-0.745577,NaN,NaN


In [1]:
df.fillna(method = 'ffill')                              #后向填充

NameError: name 'df' is not defined

In [2]:
df.fillna(method = 'ffill', limit = 2)                  #后向填充，后向填充可以连续填充最大数量为2

NameError: name 'df' is not defined

In [24]:
# fillna可以传入Series的平均值或中位数
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### 数据转换，过滤、清理以及其他的转换工作

#### 1.移除重复数据

In [25]:
# dataframe出现重复行
data = pd.DataFrame({'k1':['one','two'] * 3 + ['two'],
                    'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [26]:
# 表示各行是否是重复行 （前面出现过的行）:
data.duplicated()                                      #返回一个布尔型Series

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [27]:
data.drop_duplicates()               #返回一个DataFrame，默认针对全部列

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [28]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [29]:
data.drop_duplicates(['k1'])            #可以指定部分列进行重复项判断

,k1,k2,v1
0,one,1,0
1,two,1,1


In [30]:
data.duplicated(['k1'])                  #指定部分列进行重复项判断

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [31]:
data.drop_duplicates(['k1', 'k2'])          #默认保留的是第一个出现的值组合 (重复的行，保留第一个)

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5


In [32]:
data.drop_duplicates(['k1', 'k2'], keep='last')     #keep参数保留最后一个

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [33]:
data.duplicated(['k1', 'k2'], keep='last')          #keep参数保留最后一个

0    False
1    False
2    False
3    False
4    False
5     True
6    False
dtype: bool

#### 2.利用函数或映射进行数据转换
对于很多数据，可以根据数组、Series或DataFrame列中的值来实现转换工作

In [34]:
#肉类的数据
data = pd.DataFrame({'food':['bacon','pulled pork','bacon',
                            'Pastrami','corned beef','Bacon',
                            'pastrami','honey ham','nova lox'],
                    'ounces': [4,3,12,6,7.5,8,3,5,6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [35]:
#想要添加一列表示肉类食物来源的动物类型
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham': 'pig',
    'nova lox':'salmon'
}                                                      #编写一个不同肉类到动物的映射

In [36]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [37]:
data['animal'] = lowercased.map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [38]:
# 我们可以传入一个完成上述工作的函数
data['food'].map(lambda x: meat_to_animal[x.lower()])     #lambda 映射解决大小写不统一问题

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一种实现元素级转换及其他数据清理工作的便捷方式

#### 替换值 
fillna方法填充缺失数据可以看做值替换的一种特殊情况，map可用于修改对象的数据子集，这里用replace

In [39]:
data = pd.Series([1.,-999.,2.,-999.,-1000.,3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [40]:
#-999这个值可能是一个表示缺失数据的标记值,将其替换为NA值
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [41]:
# 一次性替换多个值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [42]:
#让每个值有不同的替换值，可以传递一个替换列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [43]:
#传入的参数也可以是字典
data.replace({-999: np.nan, -1000:0})             #inplace就地修改

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

#### 重命名轴索引
轴标签可以通过函数或映射进行转换；也可以就地修改

In [44]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                    index = ['Ohio', 'Colorado','New York'],
                    columns = ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [45]:
# 跟Series一样，轴索引也有一个map方法
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [46]:
data.index = data.index.map(transform)         #将其赋值给index，可以对DataFrame就地修改
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [47]:
# 想要创建数据集的转换板(不修改原始数据)，可以用rename
data.rename(index = str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [48]:
# rename可以结合字典型对象实现对部分轴标签的更新
data.rename(index={'OHIO':'INDIANA'},
           columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [49]:
# rename 可以实现复制DataFrame并对其索引和列标签进行赋值，就地修改
data.rename(index = {'OHIO':'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


#### 离散化和面元划分
便于分析，连续数据常常被离散化或拆分为'面元'(bin)

In [50]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)       #将ages数据划分为bins的'18-25','26-35','35-60'以及'60以上'几个面元
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

##### 含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签

In [51]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [52]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [53]:
pd.value_counts(cats)                              #pandas.cut结果的面元计数

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [54]:
# 跟‘区间’数学符号一样，方括号表示闭端(包括)，可以通过right=False修改
pd.cut(ages, [18,26,36,61,100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [55]:
pd.cut(ages, [18,26,36,61,100], right=True)

[(18, 26], (18, 26], (18, 26], (26, 36], (18, 26], ..., (26, 36], (36, 61], (36, 61], (36, 61], (26, 36]]
Length: 12
Categories (4, interval[int64]): [(18, 26] < (26, 36] < (36, 61] < (61, 100]]

In [56]:
# 可以通过传递一个列表或数组到lables，设置自己的面元名称
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [57]:
#如果传入面元数量而不是面元边界，则会根据数据的最小值和最大值计算等长面元
data = np.random.rand(20)
pd.cut(data, 4, precision=2)                     #precision = 2,限定小数只有两位

[(0.25, 0.49], (0.73, 0.97], (0.0099, 0.25], (0.25, 0.49], (0.25, 0.49], ..., (0.49, 0.73], (0.0099, 0.25], (0.25, 0.49], (0.25, 0.49], (0.0099, 0.25]]
Length: 20
Categories (4, interval[float64]): [(0.0099, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.97]]

In [58]:
#qcut是一个类似于cut的函数，根据样本分位数对数据进行面元划分，可以分得数量基本相等的面元
data = np.random.randn(1000)                        #正态分布
cats = pd.qcut(data, 4)
cats

[(-3.29, -0.611], (0.0735, 0.744], (-0.611, 0.0735], (-3.29, -0.611], (0.744, 3.114], ..., (-0.611, 0.0735], (0.744, 3.114], (-3.29, -0.611], (0.0735, 0.744], (0.0735, 0.744]]
Length: 1000
Categories (4, interval[float64]): [(-3.29, -0.611] < (-0.611, 0.0735] < (0.0735, 0.744] < (0.744, 3.114]]

In [59]:
pd.value_counts(cats)

(0.744, 3.114]      250
(0.0735, 0.744]     250
(-0.611, 0.0735]    250
(-3.29, -0.611]     250
dtype: int64

In [60]:
# qcut也可以自定义分位数(0和1之间的数值，包括端点)
pd.qcut(data, [0,0.1,0.5,0.9,1.])

[(-3.29, -1.223], (0.0735, 1.276], (-1.223, 0.0735], (-3.29, -1.223], (0.0735, 1.276], ..., (-1.223, 0.0735], (0.0735, 1.276], (-1.223, 0.0735], (0.0735, 1.276], (0.0735, 1.276]]
Length: 1000
Categories (4, interval[float64]): [(-3.29, -1.223] < (-1.223, 0.0735] < (0.0735, 1.276] < (1.276, 3.114]]

#####  qcut和cut对分组和分组分析非常重要

##### 检测和过滤异常值

In [61]:
data = pd.DataFrame(np.random.randn(1000, 4))        #正态分布的数据
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.023488,0.021158,0.000367,0.032332
std,1.012240,0.993628,0.992022,1.027269
min,-2.966936,-3.038961,-3.537477,-3.129405
25%,-0.666150,-0.669585,-0.660879,-0.689087
50%,0.035477,0.062391,0.033468,0.024231
75%,0.660484,0.707132,0.649282,0.730954
max,3.214651,2.964452,2.881308,3.330363


In [62]:
# 想找出某列中绝对值超过3的值
col = data[2]
col[np.abs(col) > 3]

14    -3.537477
992   -3.180344
Name: 2, dtype: float64

In [63]:
#  选出全部 含有'超过3或小于-3的值'的行，可以在布尔型dataframe中使用any方法
data[(np.abs(data) > 3).any()]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [64]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
14,-1.058213,-1.551555,-3.537477,-0.485149
206,0.290468,-1.369565,-0.564157,-3.129405
263,-0.519771,-1.306152,-0.348547,3.330363
594,3.214651,-0.565112,0.131178,0.959117
626,0.002446,0.020055,0.219962,3.197376
652,0.022191,0.459699,-0.289110,3.198168
705,0.816332,0.060613,-0.665811,-3.093801
935,0.622176,-3.038961,-2.208773,-1.937328
992,-0.135600,0.121205,-3.180344,1.650680


In [65]:
np.sign(data) * 3

,0,1,2,3
0,-3.0,-3.0,3.0,-3.0
1,-3.0,3.0,-3.0,-3.0
2,-3.0,3.0,-3.0,-3.0
3,3.0,3.0,3.0,-3.0
4,3.0,-3.0,-3.0,-3.0
5,-3.0,3.0,-3.0,-3.0
6,3.0,3.0,-3.0,3.0
7,3.0,-3.0,3.0,-3.0
8,3.0,3.0,-3.0,3.0
9,3.0,-3.0,-3.0,3.0


In [66]:
data.iloc[44]

0   -1.617436
1   -1.577082
2    0.819449
3    0.864452
Name: 44, dtype: float64

In [67]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.iloc[44]

0   -1.617436
1   -1.577082
2    0.819449
3    0.864452
Name: 44, dtype: float64

In [68]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,-1.0,1.0,-1.0,-1.0
3,1.0,1.0,1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


#### 排列和随机采样
numpy.random.permutation  (permuting, 随机重排序)，可以轻松实现对Series或DataFrame的排列工作

In [69]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
sampler = np.random.permutation(5)
sampler

array([2, 1, 0, 4, 3])

In [70]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [71]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


In [72]:
df.sample(n=3)                  #不想用替换的方式选取随机子集，可以用Series和DataFrame的sample方法

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


In [73]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [74]:
df.sample(n=3, replace=True)                           

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
0,0,1,2,3


In [75]:
#要通过替换的方式产生样本(允许重复选择)，可以使用replace=True
df1 = df.sample(n = 5, replace=True)
df1

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
0,0,1,2,3
0,0,1,2,3
0,0,1,2,3


In [76]:
#Series
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n = 10, replace = True)
draws

1    7
2   -1
3    6
1    7
2   -1
2   -1
0    5
4    4
4    4
1    7
dtype: int64

#### 计算指标/哑变量
将分类变量(categorical variable)转换为'哑变量'或'指标矩阵'

In [77]:
df = pd.DataFrame({'key': ['b','b','a','c','a','b'],
                  'data1':range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [78]:
pd.get_dummies(df['key'])                     #用pandas的get_dummies函数进行实现

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [79]:
#想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并
dummies = pd.get_dummies(df['key'], prefix='key')                   #prefix参数可以实现该功能
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [80]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [81]:
# 一个对统计应用有用的秘诀是: 结合get_dummies和诸如cut之类的离散化函数
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [82]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [83]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


In [84]:
# python对于大部分的文本运算都直接做成了字符串对象的内置方法
# 对于更复杂的模式匹配和文本操作，要用到正则表达式
# pandas加强了此功能，可以应用字符串表达式和正则表达式

In [85]:
# 字符串对象方法
val = 'a,b, guido'
val.split(',')                              #拆分数段

['a', 'b', ' guido']

In [86]:
pieces = [x.strip() for x in val.split(',')]            #split常与strip使用，以去除空白符(包括换行符)
pieces

['a', 'b', 'guido']

In [87]:
first, second, third = pieces
#first + '::' + second + '::' + third                    #利用加法，可以将字符串连接起来

In [88]:
#其他方法关注子串定位

'guido' in val              

True

In [89]:
val.index(',')

1

In [90]:
val.find(':')

-1

In [91]:
val.find('::')                 #find找不到会返回-1， index会报错

-1

In [92]:
val.count(',')                 #count可以返回指定子串的出现次数

2

In [93]:
# replace可以替换模式，也可以传入空 进行删除操作
val.replace(',', '::')

'a::b:: guido'

In [94]:
val.replace(',','')

'ab guido'

## 正则表达式(尽量详细全面， 暂空)

#### pandas的矢量化字符串函数

In [95]:
data = {'Dave':'dave@google.com', 'Steve':'steve@gmail.com',
       'Rob':'rob@gmail.com','Wes': np.nan}

In [96]:
data = pd.Series(data)

In [97]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [98]:
#检测是否含有缺失数据
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

#### 通过data.map,所有字符串和正则表达式都能被应用于(传入lambda表达式或其他函数)各个值，但如果存在NA就会报错，所以Series有一些能跳NA值的面向数组方法，进行字符串操作

##### Pandas的矢量化字符串函数 (不全，需要补充)

In [99]:
data.str.contains('gmail')                 #可以通过str.contains检查各个电子邮件地址是否含有'gmail'

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [100]:
import re
pattern = '([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'     #也可以使用正则表达式
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

##### 高效的数据准备可以让你将更多的时间用于数据分析，提高生产力

# 第八章 数据规整:聚合、合并和重塑

### 层次化索引，能使一个轴上拥有多个(两个以上)索引级别，即可以以低维度形式处理高维度数据

In [101]:
data = pd.Series(np.random.randn(9),
                index = [['a','a','a','b','b','c','c','d','d'],
                        [1,2,3,1,3,1,2,2,3]])
data

a  1    1.007189
   2   -1.296221
   3    0.274992
b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
d  2   -0.371843
   3    1.669025
dtype: float64

In [102]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [103]:
# 对层次化索引的对象选取子集操作
data['b']

1    0.228913
3    1.352917
dtype: float64

In [104]:
data['b':'c']

b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
dtype: float64

In [105]:
data.loc[['b','d']]

b  1    0.228913
   3    1.352917
d  2   -0.371843
   3    1.669025
dtype: float64

In [106]:
# 可以在'内层'中选取
data.loc[:, 2]

a   -1.296221
c   -2.001637
d   -0.371843
dtype: float64

In [107]:
# 层次化索引在数据重塑和基于分组的操作中也很重要
data

a  1    1.007189
   2   -1.296221
   3    0.274992
b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
d  2   -0.371843
   3    1.669025
dtype: float64

In [108]:
data.unstack()

,1,2,3
a,1.007189,-1.296221,0.274992
b,0.228913,NaN,1.352917
c,0.886429,-2.001637,NaN
d,NaN,-0.371843,1.669025


In [109]:
data.unstack().stack()

a  1    1.007189
   2   -1.296221
   3    0.274992
b  1    0.228913
   3    1.352917
c  1    0.886429
   2   -2.001637
d  2   -0.371843
   3    1.669025
dtype: float64

In [110]:
# 对于一个DataFrame,每条轴都有分层索引
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index = [['a','a','b','b'], [1, 2, 1, 2]],
                    columns = [['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [111]:
# 各层也都可以有名字
frame.index.names = ['key1','key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [112]:
# 选取列分组
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

#### 重排与分级排序 
有时候会有两种需求:1.调整某条轴上各级别的顺序，2.根据指定级别上的值对数据进行排序

In [113]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [114]:
frame.swaplevel('key1','key2')                        #互换级别

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [115]:
# sort_index根据单个级别中的值对数据进行排序
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [116]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [117]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [118]:
frame.swaplevel(0, 1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

##### 根据级别汇总统计

In [119]:
frame.sum(level = 'key2')                  #level选项

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [120]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用DataFrame的列进行索引
经常会有这样的需求: 想让一个或多个列当做行索引来用，或者想将行索引变成DataFrame的列

In [121]:
frame = pd.DataFrame({'a':range(7), 'b':range(7, 0, -1),
                     'c':['one', 'one', 'one', 'two', 'two',
                         'two','two'],
                     'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [122]:
# set_index 函数可以将一个或多个列转换为行索引
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [123]:
#默认情况，那些列会被移除，但也可以保留
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [124]:
# reset_index与set_index刚好相反，层次化索引转移到列里面
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 合并数据集（☆经常使用）
    1.pandas.merge，实现的是数据库的join操作
    2.pandas.concat可以沿一条轴将多个对象堆叠在一起
    3.实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象的缺失值

In [125]:
# 数据库风格的DataFrame合并 pd.merge
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [126]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [127]:
# 这是一种多对一的合并
pd.merge(df1, df2)                             #默认会将重叠列的列名当做键；默认是内连接（数据库）

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [128]:
# 指定用某列连接
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [129]:
df3 = pd.DataFrame({'lkey': ['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df4 = pd.DataFrame({'rkey': ['a','b','d'],'data2':range(3)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [130]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [131]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')        #指定列名进行连接

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [132]:
pd.merge(df1, df2, how='outer')                    #默认内连接，指定外连接，还有可选(left, right)

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [133]:
# 多对多
df1 = pd.DataFrame({'key': ['b','b','a','c','a','b'],
                   'data1': range(6)})
df2 = pd.DataFrame({'key': ['a','b','a','b','d'],
                   'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [134]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [135]:
pd.merge(df1, df2, on='key', how='left')                       #多对多 左连接

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [136]:
pd.merge(df1, df2, how='inner')                                #多对多，内连接

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [137]:
# 要根据多个键进行合并
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one','two','one'],
                    'lval':[1,2,3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'rval':[4,5,6,7]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [138]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [139]:
pd.merge(left, right, on=['key1','key2'],how='outer')           #根据多个键合并，并且外连接

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


#####  列-列连接时，DataFrame对象中的索引会被丢弃

In [140]:
#合并运算需要注意的一个问题 对重复列名的处理 merge有一个suffixes选项，
#指定附加到左右dataframe对象的重叠列名上的字符串
pd.merge(left, right, on ='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [141]:
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


##### 索引上的合并
    有时候连接键在索引中，可以有left_index=True/right_index=True等选项选择连接键

In [142]:
left1 = pd.DataFrame({'key': ['a','b','a','a','b','c'],
                    'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5, 7]}, index=['a','b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [143]:
right1

,group_val
a,3.5
b,7.0


In [144]:
pd.merge(left1, right1, left_on='key', right_index = True)            #左边是列，右边是索引

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [145]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')          #左边是列，右边是索引，外连接

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


##### 对层次化索引的数据进行合并

In [146]:
lefth =pd.DataFrame({'key1':['Ohio','Ohio','Ohio',
                             'Nevada','Nevada'],
                     'key2':[2000,2001,2002,2001,2002],
                     'data':np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                     index = [['Nevada','Nevada','Ohio','Ohio',
                              'Ohio','Ohio'],
                             [2001,2000,2000,2000,2001,2002]],
                     columns = [['event1', 'event2']])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [147]:
righth

event1 event2
Nevada 2001      0      1
       2000      2      3
Ohio   2000      4      5
       2000      6      7
       2001      8      9
       2002     10     11

In [148]:
# 以列表的形式指明用作合并键的多个列
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True)

,key1,key2,data,"(event1,)","(event2,)"
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [149]:
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True, how='outer')

,key1,key2,data,"(event1,)","(event2,)"
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [150]:
# 同时使用合并双方的索引
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                     index = ['a', 'c', 'e'],
                     columns = ['Ohio','Nevada'])
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14.]],
                     index = ['b','c','d','e'],
                     columns = ['Missouri', 'Alabama'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [151]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [152]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)     #合并双方索引

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### DataFrame还有一个join实例方法，它能更为方便的实现按索引合并。
#### 它可用于合并多个带有相同或相似索引的DataFrame对象
#### 但要求没有重叠的列

In [153]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [154]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [155]:
right1

,group_val
a,3.5
b,7.0


In [156]:
left1.join(right1, on='key')             # join默认使用左连接，保留左边表的行索引

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [157]:
# 对于简单的索引合并，可以向join传入一组DataFrame
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                      index = ['a', 'c', 'e','f'],
                      columns = ['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [158]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [159]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [160]:
left2.join([right2, another])                           #默认左连接，前提没有重叠列

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [161]:
left2.join([right2, another], how='outer')

D:\Anaconda\lib\site-packages\pandas\core\frame.py:6369: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


#### 轴向连接
##### 另一种数据合并运算称作连接(concatenation)、绑定(binding)或堆叠(stacking)

##### Numpy有concatenation函数

In [162]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [163]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

#####  Pandas有 concat函数

In [164]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2, 3, 4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])

In [165]:
s1

a    0
b    1
dtype: int64

In [166]:
s2

c    2
d    3
e    4
dtype: int64

In [167]:
s3

f    5
g    6
dtype: int64

In [168]:
pd.concat([s1, s2, s3])                            #默认情况是axis=0

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [169]:
s1

a    0
b    1
dtype: int64

In [170]:
s2

c    2
d    3
e    4
dtype: int64

In [171]:
s3

f    5
g    6
dtype: int64

In [172]:
pd.concat([s1, s2, s3], axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [173]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [174]:
pd.concat([s1, s4], axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [175]:
# 另外的轴上没有重叠，传入join='inner'即可得到它们的交集
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [176]:
s1

a    0
b    1
dtype: int64

In [177]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [178]:
pd.concat([s1, s4], axis=1, join_axes = [['a','c','b','e']])#join_axes指定要在其他轴上使用的索引

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [179]:
# 参与连接的片段想在结果中区分开，想要在连接轴上创建一个层次化索引，keys参数可达到这个目的
result = pd.concat([s1,s1,s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [180]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [181]:
# 沿着axis=1对Series进行合并，并设置keys参数
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [182]:
#DataFrame对象,同样逻辑

df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a','b','c'],
                  columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index = ['a', 'c'],
                  columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [183]:
df2

,three,four
a,5,6
c,7,8


In [184]:
pd.concat([df1, df2],axis=1, keys=['level1', 'level2'])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [185]:
#传入字典 ，字典的键就会被当做keys选项的值
pd.concat({'level1':df1, 'level2':df2}, axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [186]:
# names参数命名创建的轴级别
pd.concat([df1, df2], axis=1, keys = ['level1', 'level2'],
         names=['upper','lower'])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [187]:
#有种情况是 两个DataFrame不包含任何相关数据(用到ignore_index参数)
df1 = pd.DataFrame(np.random.randn(3,4), columns = ['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3), columns = ['b', 'd','a'])
df1

,a,b,c,d
0,-0.438570,-0.539741,0.476985,3.248944
1,-1.021228,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810,-0.713544


In [188]:
df2

,b,d,a
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934


In [189]:
pd.concat([df1, df2], ignore_index=True)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.438570,-0.539741,0.476985,3.248944
1,-1.021228,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810,-0.713544
3,-1.860761,-0.831154,NaN,-2.370232
4,-1.265934,-0.860757,NaN,0.560145


##### 合并重叠数据
想索引全部或部分重叠的两个数据集

In [190]:
# 举启发性的例子
# Numpy的where函数
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
             index = ['f','e','d','c','b','a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
             index=['f','e','d','c','b','a'])

In [191]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [192]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [193]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [194]:
#Series也有一个combine_first方法，实现同样功能，还带有pandas数据对齐
b[:-2].combine_first(a[2:])                   #用后者填充前者的缺失值

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [195]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [196]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [197]:
a[2:].combine_first(b[:-2])                    #用后者填充前者的缺失值

a    NaN
b    4.5
c    3.5
d    2.0
e    1.0
f    0.0
dtype: float64

In [198]:
# combine_first，用传递对象中的数据为调用对象的缺失数据‘打补丁’
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                  'b':[np.nan, 2., np.nan, 6.],
                  'c':range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                   'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [199]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [200]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 重塑和轴向旋转
    有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑(reshape)或轴向旋转(pivot)运算

#### 重塑层次化索引
    层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。主要功能有：
    1.stack:将数据的列'旋转'为行
    2.unstack:将数据的行'旋转'为列

In [201]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index = pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns = pd.Index(['one', 'two','three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [202]:
# 使用stack方法将列转行
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [203]:
# 对于一个层次化索引的Series，可以用unstack将其重排为一个DataFrame
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [204]:
#默认情况下，unstack/stack操作的是最内层，传入其他级别编号或名称也可操作
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [205]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [206]:
s1 = pd.Series([0,1,2,3], index=['a','b','c','d'])
s2 = pd.Series([4,5,6], index=['c','d','e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [207]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [208]:
# stack默认会滤除缺失数据，因此该运算是可逆的
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [209]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [210]:
data2.unstack().stack(dropna=False)                       #不过滤 缺失数据

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [211]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [212]:
df = pd.DataFrame({'left':result, 'right':result + 5},
                 columns = pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [213]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [214]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

# 将'长格式'旋转为'宽格式'
    多个时间序列数据通常是以所谓的'长格式(long)'或堆叠格式(stacked)存储在数据库和CSV中的。那样的好处是随着数据库表中的数据添加，item列中的值的种类能够增加。有的情况下，使用这样的数据会麻烦，可能更喜欢DataFrame，不同的item值分别形成一列，data列中的时间戳则作为索引，DataFrame的pivot方法可以实现这个转换。

# 时间序列数据 需要下载练习

### pivot其实就是set_index创建层次化索引，再用unstack重塑

# 将‘宽格式’旋转为‘长格式’

### 旋转DataFrame的逆运算是pandas.melt

In [215]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                  'A': [1, 2, 3],
                  'B': [4, 5, 6],
                  'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [216]:
# 使用pandas.melt, 我们必须指明哪些列是分组指标
melted = pd.melt(df, ['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [217]:
# 使用pivot,可以重塑回原来的样子
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [218]:
# pivot的结果从列中创建了一个索引，用作行标签，我们可以使用reset_index将数据移回列
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [219]:
# 可以指定列的子集，作为值的列
pd.melt(df, id_vars=['key'], value_vars = ['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [220]:
#也可以不用分组指标
pd.melt(df, value_vars = ['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [221]:
pd.melt(df, value_vars= ['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
